# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending 645-675 per student actually underperformed compared to schools with smaller budgets (585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [454]:
# Dependencies and Setup
import pandas as pd
import os
import csv

# File to Load (Remember to Change These)
school_data_to_load = "schools_complete.csv"
student_data_to_load = "students_complete.csv"
#csvpath = os.path.join("..", "Resources" , "schools_complete.csv")
#csvpath = os.path.join("..", "Resources", "students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## District Summary

In [455]:
# Calculate the total number of unique schools
school_count = len(school_data_complete["School ID"].unique())
school_count

15

In [456]:
# Calculate the total number of students
student_count = len(school_data_complete["Student ID"].unique())
student_count

39170

In [457]:
# Calculate the total budget
total_budget = school_data["budget"].sum()
total_budget

24649428

In [458]:
# Calculate the average (mean) math score
average_math_score = student_data["math_score"].mean()
average_math_score

78.98537145774827

In [459]:
# Calculate the average (mean) reading score
average_reading_score = student_data["reading_score"].mean()
average_reading_score

81.87784018381414

In [460]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [461]:
# Calculate the percentage of students who passeed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage 

85.80546336482001

In [462]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [463]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
district_results = [{"Total Schools" : school_count,
                     "Total Budget" : total_budget,
                     "Total Students" : student_count,
                     "Average Math Score" : average_math_score,
                     "Average Reading Score" : average_reading_score,
                     "Overall Passing" : overall_passing_rate,
                     "% Passing Math" : passing_math_percentage,
                     "% Passing Reading" : passing_reading_percentage}]
district_summary = pd.DataFrame(district_results)
# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["Total Schools"] = district_summary["Total Schools"].map("{:,.2f}".format)
district_summary["% Passing Math"] = district_summary["% Passing Math"].map("${:.2f}".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{:,.2f}%".format)
district_summary["% Overall Passing"] = district_summary["Overall Passing"].map("{:,.2f}%".format)

# Display the DataFrame
district_summary


,Total Schools,Total Budget,Total Students,Average Math Score,Average Reading Score,Overall Passing,% Passing Math,% Passing Reading,% Overall Passing
0,15.00,"$24,649,428.00","39,170",78.985371,81.87784,65.172326,$74.98,85.81%,65.17%


## School Summary

In [464]:
# Use the code provided to select the school type
school_types = school_data.set_index(["school_name"])["type"]

In [465]:
# Calculate the total student count
per_school_counts = school_data_complete["school_name"].value_counts()
per_school_counts

Bailey High School       4976
Johnson High School      4761
Hernandez High School    4635
Rodriguez High School    3999
Figueroa High School     2949
Huang High School        2917
Ford High School         2739
Wilson High School       2283
Cabrera High School      1858
Wright High School       1800
Shelton High School      1761
Thomas High School       1635
Griffin High School      1468
Pena High School          962
Holden High School        427
Name: school_name, dtype: int64

In [466]:
# Calculate the total school budget and per capita spending
#per_school_budget = school_data_complete.set_index('school_name')['budget']
per_school_budget = school_data_complete.groupby(['school_name']).mean()['budget']
per_school_capita = per_school_budget / per_school_counts

In [467]:
# Calculate the average test scores
per_school_math = school_data_complete.groupby(["school_name"]).mean()['math_score']
per_school_reading = school_data_complete.groupby(["school_name"]).mean()['reading_score']

In [468]:
# Calculate the number of schools with math scores of 70 or higher
school_passing_math = school_data_complete[(school_data_complete["math_score"] >= 70)]

In [469]:
# Calculate the number of schools with reading scores of 70 or higher
school_passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)] 

In [470]:
# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]

In [471]:
# Use the provided code to calculate the passing rates
pass_math = school_passing_math.groupby(['school_name']).count()['student_name']/per_school_counts *100
pass_read = school_passing_reading.groupby(['school_name']).count()['student_name']/per_school_counts *100
overall = passing_math_and_reading.groupby(['school_name']).count()['student_name']/per_school_counts *100

In [472]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame({
    "School Type" : school_types,
    "Total Students" : per_school_counts,
    "Total School Budget" : per_school_budget,
    "Per Student Budget" : per_school_capita,
    "Average Math Score" : per_school_math,
    "Average Reading Score" : per_school_reading,
    '% Passing Math' : pass_math,
    '% Passing Reading' : pass_read,
    "Overall Passing Rate" : overall
})
#per_school_summary = per_school_summary[['School Type',
                                         #'Total Students',
                                         #'Total School Budget',
                                         #'Per Student Budget',
                                         #'Average Math Score',
                                         #'Average Reading Score',
                                         #'% Passing Math',
                                         #'% Passing Reading',
                                         #'Overall Passing Rate']]

# Formatting
per_school_summary.style.format ({'Total Students' : '{:,}',
                                  "Total School Budget" : "${:,}",
                                  "Per Student Budget": "${:.0f}",
                                  'Average Math Score': "{:.1f}", 
                                  'Average Reading Score': "{:.1f}", 
                                  "% Passing Math": "{:.1%}", 
                                  "% Passing Reading": "{:.1%}", 
                                  "Overall Passing Rate": "{:.1%}"})

# Display the DataFrame
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Highest-Performing Schools (by % Overall Passing)

In [473]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(["Overall Passing Rate"], ascending=False)
top_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,1043130.0,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574.0,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [474]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(["Overall Passing Rate"], ascending= True)
bottom_schools.head(5)


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
Rodriguez High School,District,3999,2547363.0,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [475]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders =school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders =school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()

#Use the code to select only the `math_score`.
ninth_graders_math_scores = ninth_graders_scores["math_score"]
tenth_graders_math_scores = tenth_graders_scores["math_score"]
eleventh_graders_math_scores = eleventh_graders_scores["math_score"]
twelfth_graders_math_scores = twelfth_graders_scores["math_score"]

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame ({"Ninth Grade": [ninth_graders], "Tenth Grade": [tenth_graders], "Eleventh Grade": [eleventh_graders], "Twelfth Grade": [twelfth_graders]})

# Minor data wrangling
math_scores_by_grade = math_scores_by_grade[["Ninth Grade", "Tenth Grade", "Eleventh Grade", "Twelfth Grade"]]
math_scores_by_grade.index.name = "school_name"
#math_scores_by_grade.set_index("School", inplace = True)

# Display the DataFrame
math_scores_by_grade
print(ninth_graders_math_scores)

school_name
Bailey High School       77.083676
Cabrera High School      83.094697
Figueroa High School     76.403037
Ford High School         77.361345
Griffin High School      82.044010
Hernandez High School    77.438495
Holden High School       83.787402
Huang High School        77.027251
Johnson High School      77.187857
Pena High School         83.625455
Rodriguez High School    76.859966
Shelton High School      83.420755
Thomas High School       83.590022
Wilson High School       83.085578
Wright High School       83.264706
Name: math_score, dtype: float64


## Reading Score by Grade 

In [476]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()

# Use the code to select only the `reading_score`.
ninth_grade_reading_scores = ninth_graders_scores["reading_score"]
tenth_grader_reading_scores = tenth_graders_scores["reading_score"]
eleventh_grader_reading_scores = eleventh_graders_scores["reading_score"]
twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"]

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame ({"Ninth Grade": [ninth_graders], "Tenth Grade": [tenth_graders], "Eleventh Grade": [eleventh_graders], "Twelfth Grade": [twelfth_graders]})

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["Ninth Grade", "Tenth Grade", "Eleventh Grade", "Twelfth Grade"]]
reading_scores_by_grade.index.name = "school_name"

# Display the DataFrame
reading_scores_by_grade
print(ninth_grade_reading_scores)

school_name
Bailey High School       81.303155
Cabrera High School      83.676136
Figueroa High School     81.198598
Ford High School         80.632653
Griffin High School      83.369193
Hernandez High School    80.866860
Holden High School       83.677165
Huang High School        81.290284
Johnson High School      81.260714
Pena High School         83.807273
Rodriguez High School    80.993127
Shelton High School      84.122642
Thomas High School       83.728850
Wilson High School       83.939778
Wright High School       83.833333
Name: reading_score, dtype: float64


## Scores by School Spending

In [477]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [478]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [479]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=labels, right=False)
school_spending_grouped = school_spending_df.groupby("Spending Ranges (Per Student)").mean()
#school_spending_df.rename(columns={'Per Student Budget' : 'Total School Budget'}, inplace= True)
#school_spending_df.rename(columns={'Total School Budget' : 'Per Student Budget'}, inplace=True)
school_spending_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,Spending Ranges (Per Student)
Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,54.642283,$585-630
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,91.334769,<$585
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,53.204476,$630-645
Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,54.289887,$630-645
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,90.599455,$585-630
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,53.527508,$645-680
Holden High School,Charter,427,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,89.227166,<$585
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,53.539172,$645-680
Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,90.540541,$585-630


In [480]:
#  Calculate averages for the desired columns.
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_spending_rate = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Overall Passing Rate"]

#use code to select only scores




#dataframe spending_summary
spending_summary = pd.DataFrame ({
    "Average Math Score " : spending_math_scores,
    "Average Reading Score" : spending_reading_scores,
    "% Passing Math" : spending_passing_math,
    "% Passing Reading" : spending_passing_reading,
    "% Overall Passing" : overall_spending_rate
})
#spending_summary =[[
    #"Average Math Score",
    #"Average Reading Score",
    #"% Passing Math",
    #"% Passing Reading",
    #"% Overall Passing"
#]]
spending_summary.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

In [481]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [482]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.
#school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=labels, right=False)
#school_spending_grouped = school_spending_df.groupby("Spending Ranges (Per Student)").mean()

per_school_summary["School Size"]= pd.cut(per_school_counts, size_bins, labels= labels, right= False)
per_school_grouped = per_school_summary.groupby("School Size").mean()
# = school_spending_df.groupby("Total tudents").mean()
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,School Size
Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,54.642283,Large (2000-5000)
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,91.334769,Medium (1000-2000)
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,53.204476,Large (2000-5000)
Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,54.289887,Large (2000-5000)
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,90.599455,Medium (1000-2000)
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,53.527508,Large (2000-5000)
Holden High School,Charter,427,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,89.227166,Small (<1000)
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,53.539172,Large (2000-5000)
Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,94.594595,95.945946,90.540541,Small (<1000)


In [483]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["Overall Passing Rate"]

#dataframe size
per_school_summary = pd.DataFrame ({
    "Average Math Score" : size_math_scores,
    "Average Reading Score" : size_reading_scores,
    "% Passing Math" : size_passing_math,
    "% Passing Reading" : size_reading_scores,
    "Overall Passing Rate" : size_overall_passing
})
per_school_summary.head()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,83.929843,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,83.864438,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,81.344493,58.286003


In [484]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`


# Display results
#size_summary

## Scores by School Type

In [486]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = school_spending_df.groupby(["School Type"]).mean()["Average Math Score"]
type_reading_scores = school_spending_df.groupby(["School Type"]).mean()["Average Reading Score"]
type_passing_math = school_spending_df.groupby(["School Type"]).mean()["% Passing Math"]
type_passing_reading = school_spending_df.groupby(["School Type"]).mean()["% Passing Reading"]
type_overall_passing = school_spending_df.groupby(["School Type"]).mean()["Overall Passing Rate"]

 #Use the code provided to select new column data
school_spending_df = pd.DataFrame({
"Average Math Score" : type_math_scores,
"Average Reading Score" : type_reading_scores,
"% Passing Math" : type_passing_math,
"% Passing Reading" : type_reading_scores,
"% Overall Passing" : type_overall_passing
})
school_spending_df.head()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,83.896421,90.432244
District,76.956733,80.966636,66.548453,80.966636,53.672208


In [487]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame ({
"Average Math Score" : type_math_scores,
"Average Reading Score" : type_reading_scores,
"% Passing Math" : type_passing_math,
"% Passing Reading" : type_reading_scores,
"% Overall Passing" : type_overall_passing
})
# Display results
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,83.896421,90.432244
District,76.956733,80.966636,66.548453,80.966636,53.672208
